In [1]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import torch.nn.init as init
import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def seed_torch(seed=0):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

class gamma_layer(nn.Module):
    def __init__(self, input_channel, output_channel):
        super(gamma_layer, self).__init__()
        self.H = nn.Parameter(torch.ones(output_channel, input_channel))
        self.b = nn.Parameter(torch.ones(output_channel))
        self.H.data.normal_(0, 0.1)
        self.b.data.normal_(0, 0.001)

    def forward(self, x):
        H = torch.abs(self.H)
        x = F.linear(x,H)
        return torch.tanh(x)


In [13]:
intermediate_dim = 64
threshold = 0.0001
beta = 0.001
test_batch_size=1000
channel_noise_arg = 1.5
batch_size = 128
epochs = 15
lr = 0.001
gamma = 0.5
weights = 'MNIST_model_dim:64_beta:0.001_accuracy:92.4400_model.pth'

In [3]:
class gamma_function(nn.Module):

    def __init__(self):
        super(gamma_function, self).__init__()
        self.f1 = gamma_layer(1,16)
        self.f2 = gamma_layer(16,16)
        self.f3 = gamma_layer(16,16)
        self.f4 = gamma_layer(16,intermediate_dim)
        
    def forward(self, x):
        x = self.f1(x)
        x = self.f2(x)
        x = self.f3(x)
        x = self.f4(x)
        return x

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(784, intermediate_dim)
        self.fc2 = nn.Linear(intermediate_dim, 1024)
        self.fc2_2 = nn.Sequential(
                        nn.Linear(1,16),
                        nn.ReLU(),
                        nn.Linear(16,16),
                        nn.ReLU(),
                        nn.Linear(16,16),
                        nn.ReLU()
                        )
        self.fc3 = nn.Linear(1024 + 16, 256)
        self.fc4_reduced = nn.Linear(256, 6)  # Classifier for reduced labels
        self.fc4_final = nn.Linear(256, 10)  # Classifier for final labels
        self.gamma_mu = gamma_function().to(device)
        self.upper_tri_matrix = torch.triu(torch.ones((intermediate_dim, intermediate_dim))).to(device)
        self.mask_matrix = create_mask_matrix().to(device)

    def get_mask(self, mu, threshold=threshold):
        hard_mask = (mu > threshold).float()
        return hard_mask

    def get_mask_test(self, channel_noise, threshold=threshold):
        mu = self.gamma_mu(channel_noise)
        alpha = F.linear(mu, self.upper_tri_matrix)
        mu = torch.clamp(mu, min=1e-4)
        hard_mask = (alpha > threshold).float()
        return hard_mask, alpha

    def forward(self, x, noise=0.2):
        x = x.view(-1, int(x.nelement() / x.shape[0]))

        weight = self.fc1.weight
        bias = self.fc1.bias
        l2_norm_squared = torch.sum(weight.pow(2), dim=1) + bias.pow(2)
        l2_norm = l2_norm_squared.pow(0.5)
        fc1_weight = (weight.permute(1, 0) / l2_norm).permute(1, 0)
        fc1_bias = bias / l2_norm
        x = F.linear(x, fc1_weight, fc1_bias)

        # Dynamic Channel Conditions
        if self.training:
            b = torch.bernoulli(1/5.0*torch.ones(1))
            if b > 0.5:
                channel_noise = torch.ones(1) * 0.3162
            else:
                channel_noise = torch.rand(1) * 0.27 + 0.05
        else:
            channel_noise = torch.FloatTensor([1]) * noise
        channel_noise = channel_noise.to(device)
        noise_feature = self.fc2_2(channel_noise)
        noise_feature = noise_feature.expand(x.size()[0], 16)
        mu = self.gamma_mu(channel_noise)
        mu = F.linear(mu, self.upper_tri_matrix)
        mu = torch.clamp(mu, min=1e-4)
        x = torch.tanh(mu * x)
        KL = self.KL_log_uniform(channel_noise**2 / (x.pow(2) + 1e-4))

        if self.training:
            x = (x * self.get_mask(mu) - x).detach() + x
            x = x + torch.randn_like(x) * channel_noise * self.get_mask(mu)
        else:
            x = x + torch.randn_like(x) * channel_noise
            x = x * self.get_mask(mu)

        x = F.relu(self.fc2(x))
        x = torch.cat((x, noise_feature), dim=1)
        x = F.relu(self.fc3(x))
        
        output_reduced = self.fc4_reduced(x)  # First stage output
        output_final_logits = self.fc4_final(x)  # Second stage logits

        return F.log_softmax(output_reduced, dim=1), output_final_logits, KL * (0.1 / channel_noise)

    def KL_log_uniform(self, alpha_squared):
        k1 = 0.63576
        k2 = 1.8732
        k3 = 1.48695
        batch_size = alpha_squared.size(0)
        KL_term = k1 * torch.sigmoid(k2 + k3 * torch.log(alpha_squared)) - 0.5 * F.softplus(-torch.log(alpha_squared)) - k1

        return -torch.sum(KL_term) / batch_size


In [5]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output_reduced, output_final_logits, KL = model(data)
        
        # Get reduced labels
        target_reduced = get_reduced_labels(target)
        
        # Compute the final output based on the reduced label mask
        batch_size = output_final_logits.size(0)
        mask = model.mask_matrix[target_reduced]
        masked_logits = output_final_logits * mask
        
        # Normalize masked logits to form a proper log_softmax
        output_final = F.log_softmax(masked_logits, dim=1)
        
        if epoch <= 5:
            loss_reduced = F.nll_loss(output_reduced, target_reduced)
            loss_final = F.nll_loss(output_final, target)
        else:
            anneal_ratio = min(1, (epoch - 5) / 10)
            loss_reduced = F.nll_loss(output_reduced, target_reduced)
            loss_final = F.nll_loss(output_final, target) + beta * KL * anneal_ratio
        
        loss = loss_reduced + loss_final
        loss.backward()
        optimizer.step()

def get_reduced_labels(target):
    # Define the mapping from original labels to reduced labels
    reduced_labels = target.clone()
    label_mapping = {
        0: 0,  # T-shirt/top
        1: 1,  # Trouser
        2: 2,  # Pullover
        3: 3,  # Dress
        4: 2,  # Coat
        5: 5,  # Sandal
        6: 0,  # Shirt
        7: 5,  # Sneaker
        8: 4,  # Bag
        9: 5   # Ankle boot
    }
    
    for original, reduced in label_mapping.items():
        reduced_labels[target == original] = reduced
        
    return reduced_labels

In [6]:

def create_mask_matrix():
    mask_matrix = torch.zeros((6, 10))  # 6 reduced labels, 10 final labels
    # Define the valid final labels for each reduced label
    mask_matrix[0, [0, 6]] = 1  # Reduced label 0 maps to final labels 0 and 6
    mask_matrix[1, [1]] = 1     # Reduced label 1 maps to final label 1
    mask_matrix[2, [2, 4]] = 1  # Reduced label 2 maps to final labels 2 and 4
    mask_matrix[3, [3]] = 1     # Reduced label 3 maps to final label 3
    mask_matrix[4, [8]] = 1     # Reduced label 4 maps to final label 8
    mask_matrix[5, [5, 7, 9]] = 1  # Reduced label 5 maps to final labels 5, 7, 9
    return mask_matrix

mask_matrix = create_mask_matrix().to(device)

In [7]:
def test(model, device, test_loader, noise=0.2):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output_reduced, output_final_logits, KL = model(data, noise=noise)

            # Get reduced labels
            target_reduced = get_reduced_labels(target)

            # Compute the final output based on the reduced label mask
            batch_size = output_final_logits.size(0)
            mask = model.mask_matrix[target_reduced]
            masked_logits = output_final_logits * mask

            # Normalize masked logits to form a proper log_softmax
            output_final = F.log_softmax(masked_logits, dim=1)

            test_loss += F.nll_loss(output_final, target, reduction='sum').item()
            pred = output_final.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            # print("Actual labels:", target.cpu().numpy())
            # print("Reduced labels:", target_reduced.cpu().numpy())
            # print("Predicted labels:", pred.cpu().numpy().flatten())

    test_loss /= len(test_loader.dataset)
    hard_mask, mu = model.get_mask_test(torch.FloatTensor([noise]).to(device))
    index = torch.nonzero(torch.lt(hard_mask, 0.5)).squeeze(1)
    pruned_number = index.size()[0]

    return 100. * correct / len(test_loader.dataset), pruned_number


In [8]:
def main_train():
    kwargs = {'num_workers': 1, 'pin_memory': True}
    test_loader = torch.utils.data.DataLoader(datasets.FashionMNIST('./data', train=False, download=True, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.5,), (0.5,))
                       ])),
        batch_size=test_batch_size, shuffle=True, **kwargs)

    model = Net().to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay = 5e-5)
    scheduler = StepLR(optimizer, step_size=45, gamma=gamma)
    
    test_acc = 0
    pruned_dim = 0
    saved_model = {}

    for epoch in range(1, epochs + 1):
        if epoch % 10 == 1:
            train_loader = torch.utils.data.DataLoader(
                datasets.FashionMNIST('./data', train=True, download=True,
                               transform=transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5,), (0.5,))
                               ])),
                batch_size=batch_size, shuffle=True, **kwargs)
        print('\nepoch:',epoch)
        train(model, device, train_loader, optimizer, epoch)
        scheduler.step()
        accuracy = 0
        t = 5
        for i in range (t):
            acc, pruned_number = test(model, device, test_loader, channel_noise_arg)
            accuracy += acc
        print('Test Accuracy:',accuracy/t, 'Pruned dim',pruned_number,'Activated dim:',intermediate_dim - pruned_number)
        accuracy = accuracy/t

        if epoch > 10:
            if (accuracy > test_acc and pruned_number == pruned_dim) or pruned_number > pruned_dim:
                test_acc = accuracy
                pruned_dim = pruned_number
                saved_model = copy.deepcopy(model.state_dict())
    print('Best Accuray:',test_acc,'pruned_number:',pruned_dim,'activated_dim:',intermediate_dim - pruned_dim)
    torch.save({'model': saved_model}, './MNIST_model_dim:{}_beta:{}_accuracy:{:.4f}_model.pth'.format(intermediate_dim - pruned_dim, beta, test_acc))


In [9]:
def main_test():
    kwargs = {'num_workers': 1, 'pin_memory': True}
    test_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST('./data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.5,), (0.5,))
                       ])),
        batch_size=test_batch_size, shuffle=True, **kwargs)

    model = Net().to(device)
    model.load_state_dict(torch.load(weights, map_location=torch.device('cpu'))['model'])

    accuracy = 0
    t = 20
    for i in range (t):
        acc, pruned_dim = test(model, device, test_loader, channel_noise_arg)
        accuracy += acc
    print('Noise level:',channel_noise_arg, 'Test Accuracy:', accuracy/t, 'Pruned dim:', pruned_dim, 'Activated dim:', intermediate_dim - pruned_dim)


In [14]:
main_test()

Noise level: 1.5 Test Accuracy: 80.93900000000001 Pruned dim: 0 Activated dim: 64
